In [1]:
import sqlite3
import collections
import json
import re
from JMDictModule import JMDictPOS

In [2]:
data = open('star_nhatviet.tab','r',encoding="utf-8").read()
sentences = data.split("\n")

In [3]:
kanjiStart = 87283

In [4]:
class Token:
    def __init__(self,word,reading,pos,desc,need_translation = False):
        self.word = word
        self.reading = reading
        self.pos = pos
        self.desc = desc
        self.need_translation = need_translation
    def __str__(self):
        return f"word {self.word} reading {self.reading} pos {self.pos} desc {self.desc}"
    
    def to_dict(self):
        return {
            "word": self.word,
            "reading": self.reading,
            "pos": self.pos,
            "desc": self.desc,
            "need_translation": self.need_translation,
        }

In [5]:
def contains_english(text: str) -> bool:
    return bool(re.search(r'[A-Za-z]', text))

In [6]:
dictionary = JMDictPOS("JMdict_e")

In [78]:
print(dictionary.lookup("食べる"))
print(dictionary.lookup("あの世"))

{'pos': ['Ichidan verb', 'transitive verb', 'Ichidan verb', 'transitive verb'], 'readings': ['たべる']}
{'pos': ['noun (common) (futsuumeishi)'], 'readings': ['あのよ']}


process kanji

In [62]:
results = []
first_pass_exception = []
for line in sentences[kanjiStart-1:-1]:
    if "@" in line or "*" in line:
        first_pass_exception.append(line)
        continue
    try:
        key, desc = line.split("\t-",1)
        key = key.strip()
        desc = desc.removesuffix(",")
        dict_look_up = dictionary.lookup(key)
        pos = dict_look_up['pos']
        if "," in desc:
            readings, meaning = desc.split(",",1)
            readings = readings.split(" - ")
            jap_reading = []
            english_meanings = []
            for i in readings:
                i = i.strip().removeprefix("{").removesuffix("}")
                if contains_english(i):
                    english_meanings.append(i)
                else:
                    jap_reading.append(i)
            results.append(Token(key,jap_reading,pos , [meaning]))
        else:
            readings = desc.removeprefix("-").removesuffix("-").split(" - ")
            readings = list(filter(lambda i: i!= "", readings))
            jap_reading = []
            english_meanings = []
            for i in readings:
                i = i.strip().removeprefix("{").removesuffix("}")
                if contains_english(i):
                    english_meanings.append(i)
                else:
                    jap_reading.append(i)
            if len(jap_reading) == 0:
                print(line)
                jap_reading = dict_look_up['reading']
                print(jap_reading)
            token = Token(key,jap_reading,pos, english_meanings,need_translation=True)
            results.append(token)
    except:
        first_pass_exception.append(line)


一人暮らし	- sống độc thân - sống cô đơn không người thân - cuộc sống tự lập của học sinh học xa nhà
下向き	- Hướng xuống dưới - Suy nghĩ theo chiều hướng tiêu cực - (kinh tế) sụt giảm
下宿屋	- nhà trọ.
下旬	- hạ tuần (10 ngày cuối của một tháng).
下車する	- xuống xe.
不便な	- bất tiện.
不健康	- tướng không khỏe. - trạng thái ốm yếu.
不具者	- người què - người tàn tật
不動産	- bất động sản (= nhà đất).
不孝な	- bất hiếu.
不平のある	- không bằng lòng.
不幸な	- (một cách) bất hạnh / không may / thê thảm.
不意打ち	- sự tập kích (sự tấn công bất ngờ).
不意打ちをくわす	- tấn công bất ngờ.
不衛生な	- mất vệ sinh.
不通になる	- bị gián đoạn.
不遇な	- không may mắn.
不運にも	- một cách không may.
不道徳な	- mất đạo đức.
両為	-có lợi cho cả hai bên
中世	- thời Trung cổ.
中元	- lễ Trung nguyên (rằm tháng 7).
中国	- Trung Quốc.
中国料理	- thức ăn Trung Quốc.
中国語	- tiếng Trung Quốc
中央	- trung tâm.
中年	- trung niên
中年の人	- người trung niên.
中毒	- sự trúng độc.
中毒する	- bị trúng độc
中産階級	- giai cấp trung lưu.
中立	- sự trung lập.
中立の	- trung lập.
中間テスト	- Thi giữa kỳ
中間報告	- Báo cáo giữa kỳ


In [60]:
first_pass_exception

['ー  - {ちょうおん} -{long vowel mark (usually only used in katakana)}',
 '一人暮らし\t- sống độc thân - sống cô đơn không người thân - cuộc sống tự lập của học sinh học xa nhà',
 '三日月\t@みかずき - Trăng non, trăng thượng tuần, trăng lưỡi liềm',
 '下向き\t- Hướng xuống dưới - Suy nghĩ theo chiều hướng tiêu cực - (kinh tế) sụt giảm',
 '下宿する\t* động từ - ở trọ, tạm trú.',
 '下宿屋\t- nhà trọ.',
 '下旬\t- hạ tuần (10 ngày cuối của một tháng).',
 '下車する\t- xuống xe.',
 '不便\t- sự bất tiện, sự phiền phức @ふびん - {pity} , lòng thương hại, lòng thương xót, lòng trắc ẩn, điều đáng thương hại, điều đáng tiếc, thương hại, thương xót, động lòng trắc ẩn đối với - {compassion} , lòng thương, lòng trắc ẩn',
 '不便な\t- bất tiện.',
 '不健康\t- tướng không khỏe. - trạng thái ốm yếu.',
 '不具\t@ふぐ - dị hình, tật nguyền - chân què.',
 '不具者\t- người què - người tàn tật',
 '不動産\t- bất động sản (= nhà đất).',
 '不孝な\t- bất hiếu.',
 '不平のある\t- không bằng lòng.',
 '不幸な\t- (một cách) bất hạnh / không may / thê thảm.',
 '不意打ち\t- sự tập kích (sự

In [23]:
for token in results[0:10]:
    print(token.reading)

['かげつ']
['くりかえし']
['くりかえし']
['いち']
['いちいち', '{いついつ']
['いちからじゅうまで']
['いちかばちか']
['いっかしょ']
['いっかげつ']
['ひとっぱしり']


In [ ]:
second_pass_exception = []
for line in first_pass_exception:
    try:
        if "\t@" in line:
            word, desc = line.split("\t@",1)
            reading, meaning = line.split("-",1)
            reading = reading.strip().removeprefix("{").removesuffix("}")
            word = word.strip().removeprefix("{").removesuffix("}")
            pos = dictionary.lookup(word)['pos']
            results.append(Token(word=word, reading=reading,pos=pos,desc=meaning))
            continue
        if "\t*" in line:
            word, desc = line.split("\t*",1)
            wtype,meaning = line.split("-",1)
            word = word.strip().removeprefix("{").removesuffix("}")
            dict_look_up = dictionary.lookup(word)

            results.append(Token(word=word,reading=dict_look_up['readings'], pos=dict_look_up['pos'],desc=meaning))
            continue
        second_pass_exception.append(line)
    except:
        second_pass_exception.append(line)

In [47]:
for entry in results:
    if "安産" in entry.word:
        print(entry)

word 安産 reading [] pos ['noun (common) (futsuumeishi)', 'noun or participle which takes the aux. verb suru', 'transitive verb', 'intransitive verb'] desc ['Sự sinh đẻ dễ dàng =安産する(あんさんする)']


In [ ]:
second_pass_exception

In [48]:
results[0].pos

['counter']

In [42]:
manual = [
    Token(word="不便", reading=["ふびん"], pos="Noun", desc=["lòng thương hại", "lòng thương xót", "lòng trắc ẩn", "điều đáng thương hại", "điều đáng tiếc", "thương hại", "thương xót", "compassion"]),
    Token(word="便", reading=["よすが", "びん"], pos="Noun", desc=["sự tiện lợi", "sự thuận tiện", "đường", "lối đi", "tiền bạc", "của cải", "phương tiện", "biện pháp", "mail", "thư từ", "bưu kiện", "post", "vị trí", "chức vụ", "flight", "service", "sự phục vụ", "opportunity", "cơ hội", "chance", "sự may rủi", "letter", "thư"]),
    Token(word="分", reading=["ぶん", "ぶ", "ふん"], pos="Noun", desc=["part", "phần", "bộ phận", "đoạn", "khúc", "share", "cổ phần", "ration", "khẩu phần", "rate", "tỷ lệ", "giá", "degree", "mức độ", "one's lot", "one's status", "relation", "duty", "bổn phận", "kind", "loài", "lot", "số phận", "in proportion to", "just as much as", "percentage", "tỷ lệ phần trăm", "one percent", "thickness", "độ dày", "odds", "chance of winning", "phút"]),
    Token(word="分別", reading=["ふんべつ"], pos="Noun", desc=["sự phân biệt (phải, trái)", "sự phán đoán", "sự biết điều", "discernment", "sự nhận thức rõ", "sự sâu sắc", "discretion", "sự thận trọng", "sự suy xét khôn ngoan", "judgement", "sự xét xử", "quyết định của toà", "óc phán đoán", "discrimination", "sự đối xử phân biệt"]),
    Token(word="安心", reading=["あんしん"], pos="Noun", desc=[ "sự an tâm", "sự yên trí"]),
    Token(word="揶揄う", reading=["からかう"], pos="Verb", desc=["trêu chọc", "chế nhạo", "giễu cợt"]),
    Token(word="節", reading=["ふし", "せち", "さだめ", "せつ", "たかし", "まこと", "みさお"], pos="Noun", desc=["Đoạn", "khúc", "Mắt", "nút", "khớp", "đốt (tre)", "Mùa", "thời tiết", "tên riêng"]),
    Token(word="米印", reading=["こめじるし"], pos="Noun", desc=["asterisk", "dấu hoa thị (*)"]),
    Token(word="脂", reading=["あぶら"], pos="Noun", desc=["nicôtin", "nhựa (cây)", "mỡ", "chất béo"]),
    Token(word="追放する", reading=["ついほうする"], pos="Verb", desc=["trục xuất", "truy방", "đày ải"]),
    Token(word="額", reading=["ひたい", "がく"], pos="Noun", desc=[ "trán", "lông mày", "tấm bảng", "khung tranh", "số tiền"])
]
for token in manual:
    token.pos = dictionary.lookup(token.word)['pos']
    results.append(token)


In [43]:
results[100].pos
len(results)

85023

process hiragana

In [44]:
processed = set()
for entry in results:
    for reading in entry.reading:
        if reading not in processed:
            processed.add(reading)

In [45]:

for i in processed:
    if "あだな" in i:
        print(i)

"あだな" in processed

あだな


True

In [12]:
hiragana_end = 69417
hiragana_start = 12

In [30]:
first_pass_exception = []
for line in sentences[hiragana_start: hiragana_end]:
    try:
        word,desc = line.split("\t",1)
        if word.strip() in processed:
            continue
        first_pass_exception.append(line)
    except:
        first_pass_exception.append(line)

In [35]:
first_pass_exception

['ああ言えばこう言う\t- {ああいえばこういう} - {rationalize} , hợp lý hoá (một ngành sản xuất), giải thích duy lý (tín ngưỡng...) làm cho (hành vi tư tưởng...) phù hợp với lẽ phải, (toán học) hữu tỷ hoá, theo chủ nghĩa duy lý, hành động suy nghĩ theo chủ nghĩa duy lý',
 'あいさつする\t* động từ - chào, chào hỏi',
 'あいする\t- {愛する} - {love} , lòng yêu, tình thương, tình yêu, mối tình, ái tình, người yêu, người tình, thần ái tình, (thông tục) người đáng yêu; vật đáng yêu, (thể dục,thể thao) điểm không, không (quần vợt), ái tình và nước lã, không có cách gì lấy được cái đó, chơi vì thích không phải vì tiền, chúng nó ghét nhau như đào đất đổ đi, yêu, thương, yêu mến, thích, ưa thích',
 'あいず\t- {合図} - {sign} , dấu, dấu hiệu, ký hiệu; mật hiệu, dấu hiệu biểu hiện, biểu hiện, tượng trưng, điểm, (y học) triệu chứng, dấu vết (thú săn...), biển hàng, ước hiệu, gọi gió làm mưa, hô phong hoán vũ, đường như là chết, không phản ứng gì, đánh dấu; (tôn giáo) làm dấu, ký tên, viết ký hiệu, viết dấu hiệu, ra hiệu, làm hiệu, nhườ

In [36]:
second_pass_exception = []
more_than_one_a = []
for line in first_pass_exception:
    if "@" in line:
        if line.count("@") > 1:
            more_than_one_a.append(line)
        print(line)

あんざん	Xem 安産 @暗算 Xem 暗算
ちり	- khoa địa lý; địa lý học @塵 - - bụi. - rác rưởi, đồ phế thải.


word 安産 reading [] pos ['noun (common) (futsuumeishi)', 'noun or participle which takes the aux. verb suru', 'transitive verb', 'intransitive verb'] desc ['Sự sinh đẻ dễ dàng =安産する(あんさんする)']


In [65]:
import json

output_filename = "tokens_output.json"

try:
    with open(output_filename, "w", encoding="utf-8") as f:
        # Dump results as JSON (pretty printed)
        json.dump([t.to_dict() for t in results], f, ensure_ascii=False, indent=2)

    print(f"Successfully wrote {len(results)} tokens to '{output_filename}'")

except IOError as e:
    print(f"An error occurred while writing to the file: {e}")


Successfully wrote 85023 tokens to 'tokens_output.json'
